# Bitcoin price prediction using LSTM


In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout,GRU
from tensorflow.keras.optimizers import Adam


## Read Data

In [ ]:
df = pd.read_csv("data/btc.csv")

df = df.set_index(pd.DatetimeIndex(df.iloc[:,0].values)).iloc[:,1:]

df

In [ ]:
fig = go.Figure(
data = [
    go.Candlestick(
    x = df.index,
        low = df['low'],
        high = df['high'],
        close = df['close'],
        open = df['open'],
        increasing_line_color = "green",
        decreasing_line_color = "red")])

fig.update_layout(
    yaxis_title = "Price",
    xaxis_title = "Date"
)

fig.show()

## Data Preparation

### Normalization

In [ ]:
data = df[['close','volume','open','low','high']]
num_features = data.shape[1]

In [ ]:
scaler = MinMaxScaler()
norm_data = scaler.fit_transform(data.values)
print("Real: {}\nNormalized: {}".format(data.values[0],norm_data[0]))

### Data split

In [ ]:
past_history = 10
future_target = 0
split_rate = int(len(norm_data) * 0.8) # 80 percent

In [ ]:
input_data = []
output_data = []

for i in range(past_history, len(norm_data)+1): 
    indices = range(i-past_history, i)

    input_data.append(np.reshape(norm_data[indices], (past_history, num_features)))
    try:
        output_data.append(norm_data[i+future_target][0])
    except:
        output_data.append(np.nan)

input_data,output_data = np.array(input_data), np.array(output_data)

In [ ]:
x_train, x_test = input_data[:split_rate], input_data[split_rate:]
y_train, y_test = output_data[:split_rate], output_data[split_rate:]

## Build the model

In [ ]:
num_units = 100
activation_function = 'relu'
loss_function = 'mean_absolute_error'
batch_size = 32
num_epochs = 100

model = Sequential()

model.add(LSTM(units = num_units,return_sequences=True,activation=activation_function, input_shape=(None, num_features)))
model.add(Dropout(0.1))

model.add(LSTM(units=num_units,activation=activation_function))
model.add(Dropout(0.1))

model.add(Dense(units = 1))

model.compile(optimizer=Adam(learning_rate=0.0001), loss=loss_function)

In [ ]:
model.summary()

## Train the model

In [ ]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=num_epochs,
    shuffle=False
)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()

plt.plot(range(len(loss)), loss, 'blue', label='Training loss')
plt.plot(range(len(loss)), val_loss, 'red', label='Validation loss')
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.legend()

plt.show()

In [ ]:
original_train = y_train - scaler.min_[0]
original_train /= scaler.scale_[0]
original_train = pd.DataFrame((original_train))

predictions_train = model.predict(x_train) - scaler.min_[0]
predictions_train /= scaler.scale_[0]
predictions_train = pd.DataFrame((predictions_train))

plt.figure()

plt.plot(original_train, 'blue', label='Train Data')
plt.plot(predictions_train, 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()
print("Train error:")
print(mean_absolute_error(y_train, model.predict(x_train)))

## Prediction

In [ ]:
original_test = y_test - scaler.min_[0]
original_test /= scaler.scale_[0]
original_test = pd.DataFrame((original_test))

predictions_test = model.predict(x_test) - scaler.min_[0]
predictions_test /= scaler.scale_[0]
predictions_test = pd.DataFrame((predictions_test))

plt.figure()

plt.plot(original_test[:-1], 'blue', label='Test Data')
plt.plot(predictions_test[:-1], 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()
print("Test error:")
print(mean_absolute_error(y_test[:-(future_target+1)], model.predict(x_test)[:-(future_target+1)]))

In [ ]:
prediction = pd.DataFrame({"actual price":original_test[0],"predicted":predictions_test[0]})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(prediction)
